<a href="https://colab.research.google.com/github/AugustinCablant/Viager/blob/main/Viager_init.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Code Stage Augustin Cablant (2023)**

> Ce document regroupe les lignes de code utilisées pour mon stage.

## Imports

In [1]:
!pip install reportlab
!pip install pdfkit
!pip install tqdm
!pip install dateparser
!pip install unidecode
!pip install Levenshtein

import pandas as pd
import numpy as np
from datetime import datetime
import locale
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.platypus import SimpleDocTemplate, Table
import pdfkit
import tabulate
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import html
import dateparser
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Stage')
from tqdm import tqdm
from unidecode import unidecode
import Levenshtein
from nltk.metrics.distance import jaccard_distance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 70.0 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 12.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 14.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.4 MB/s eta 0:00:00
Mounted at /content/drive


# Phase 1 : création de la base de travail

## DATA FRAME INSEE

> Ce Data Frame regroupe les fichiers de l'INSEE recensant les morts de 2000 à 2022.

> Ce Data Frame regroupe les fichiers de l'INSEE recensant les morts de 1984 à 1999.

> Il suffit de modifier la boucle for ...



In [149]:
dossier = '/content/drive/MyDrive/Stage/sas/sas/'
# Liste pour stocker les DataFrames
dataframes = []

# Boucle pour créer ma liste de DataFrames

for i in tqdm(range(2000,2023)):
    # Obtenir la liste des fichiers CSV dans le dossier
    chemins_fichiers_sas = dossier+f'a{i}.sas7bdat'
    dataframes.append(pd.read_sas(chemins_fichiers_sas))

df_ins = pd.concat(dataframes)
n = df_ins.shape[0]
df_ins['index']= list(range(1,n+1))
df_ins.set_index('index', inplace=True)

# Data Frame Insee avec les bons noms de colonne et les bons formats
df_insee1 = pd.DataFrame(index=list(range(1,df_ins.shape[0])), columns = ['nom', 'prenoms',
        'sexe', 'ybirth', 'mbirth', 'dbirth','villebirth', 'paysbirth', 'Dpt_naissance',
        'ydeath', 'mdeath', 'ddeath', 'codebirth', 'codebirth', 'nbdeath', 'type', 'reserve', 'fichier', 'allobs'])


df_insee1['nom'] = df_ins['nom'].str.decode('utf-8', 'ignore')[2:-1]
df_insee1['prenoms'] = df_ins['prenoms'].str.decode('utf-8', 'ignore')[2:-1]


df_insee1['ybirth'] = df_ins['ybirth']
df_insee1['mbirth'] = df_ins['mbirth']
df_insee1['dbirth'] = df_ins['dbirth']
df_insee1['ydeath'] = df_ins['ydeath']
df_insee1['mdeath'] = df_ins['mdeath']
df_insee1['ddeath'] = df_ins['ddeath']
df_insee1['Dpt_naissance'] = (df_ins['codebirth'].str.decode('utf-8')).str.slice(0,2)
df_insee1['paysbirth'] = df_ins['paysbirth'].str.decode('iso-8859-1')
df_insee1['villebirth'] = df_ins['villebirth'].str.decode('iso-8859-1')
df_insee1['codebirth'] = df_ins['codebirth']
df_insee1['codebirth'] = df_ins['codebirth']
df_insee1['nbdeath'] = df_ins['nbdeath']
df_insee1['type'] = df_ins['type']
df_insee1['reserve'] = df_ins['reserve']
df_insee1['fichier'] = df_ins['fichier']
df_insee1['allobs'] = df_ins['allobs']

100%|██████████| 23/23 [02:25<00:00,  6.33s/it]


In [ ]:
df_insee1['sexe'] = df_ins['sex']
df_insee1['Dpt_naissance']= pd.to_numeric(df_insee1['Dpt_naissance'], errors='coerce').astype(float)

## DATA FRAME CLIENT

In [14]:
df_grille = pd.read_csv('data_agence.csv', encoding='ISO-8859-1')

def convert_to_timestamp(value):
    if pd.isnull(value) or value == '/':
        return value
    else:
        try:
            timestamp = dateparser.parse(str(value))
            return timestamp
        except ValueError:
            return value

df_grille['Date_naissance_vendeur_1'] = df_grille['Date_naissance_vendeur_1'].apply(convert_to_timestamp)
df_grille['Date_naissance_vendeur_2'] = df_grille['Date_naissance_vendeur_2'].apply(convert_to_timestamp)
df_grille = df_grille.dropna(subset = 'Tete_1')
df_grille['Tete_1'] = df_grille['Tete_1'].astype(str)
df_grille['Tete_2'] = df_grille['Tete_2'].astype(str)
df_grille[df_grille['Tete_1']=='Vendeur 1']['lieutete1'] = df_grille[df_grille['Tete_1']=='Vendeur 1']['Ville_naissance_vendeur_1']
df_grille[df_grille['Tete_1']=='Vendeur 2']['lieutete1'] = df_grille[df_grille['Tete_1']=='Vendeur 2']['Ville_naissance_vendeur_2']
df_grille[df_grille['Tete_2']=='Vendeur 2']['lieutete2'] = df_grille[df_grille['Tete_2']=='Vendeur 2']['Ville_naissance_vendeur_2']

<ipython-input-14-6e36dea75606>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grille['Tete_1'] = df_grille['Tete_1'].astype(str)
<ipython-input-14-6e36dea75606>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_grille['Tete_2'] = df_grille['Tete_2'].astype(str)
<ipython-input-14-6e36dea75606>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.py

In [15]:
colonnes = ['Adresse_bien_vendu', 'Prenom_vendeur_1', 'Nom_vendeur_1',
       'Nom_vendeur_2', 'Nom_acheteur_2', 'Type_de_viager', 'Majoration',
       'Bouquet', 'Rente', 'Tete_1', 'Tete_2', 'Commission', 'Milliemes',
       'Standing', 'Remarques', 'Valeur_libre', 'Valeur_occupee', 'Numero',
       'Repertoire', 'Prenom_vendeur_2', 'Date_naissance_vendeur_1',
       'Ville_naissance_vendeur_1', 'Departement_naissance_vendeur_1',
       'Pays_naissance_vendeur_1', 'Date_naissance_vendeur_2',
       'Ville_naissance_vendeur_2', 'Departement_naissance_vendeur_2',
       'Pays_naissance_vendeur_2', 'Adresse_vendeur', 'Nom_acheteur_1',
       'Profession_acheteur_1', 'Date_naissance_acheteur_1',
       'Profession_acheteur_2', 'Date_naissance_acheteur_2',
       'Adresse_acheteur', 'Indexation_rente', 'Date_acte_authentique',
       'Payeur_Com', 'Chambre_de_bonnes', 'Nb_pieces', 'Etage',
       'Mettres_carres', 'Ascenseur', 'Parking_Box', 'Cave', 'Date_immeuble',
       'Pierre_de_taille', 'source', 'Ch_de_service', 'Parking', 'Bonus',
       'annee', 'type', 'nom1', 'nom2', 'prenom1', 'prenom2', 'sexe1', 'sexe2',
       'annee1', 'annee2', 'naissancetete1', 'mois1', 'jour1',
       'naissancetete2', 'mois2', 'jour2', 'lieutete1', 'dep1', 'lieutete2',
       'dep2', 'abroad1', 'abroad2', 'quality', 'nom', 'justenom1', 'societe',
       'vsociete', 'asociete', 'legasse', 'nblegassenet', 'reprise', 'limite',
       'stranger1', 'euro', 'franc', 'downp', 'annuity', 'VL', 'VO', 'done1']

colonnes1 = ['nom1', 'prenom1', 'annee1', 'mois1', 'jour1',
       'dep1', 'abroad1','lieutete1','sexe1',
        'Adresse_bien_vendu', 'Prenom_vendeur_1', 'Nom_vendeur_1',
       'Nom_vendeur_2', 'Nom_acheteur_2', 'Type_de_viager', 'Majoration',
       'Bouquet', 'Rente', 'Tete_1', 'Tete_2', 'Commission', 'Milliemes',
       'Standing', 'Remarques', 'Valeur_libre', 'Valeur_occupee', 'Numero',
       'Repertoire', 'Prenom_vendeur_2', 'Date_naissance_vendeur_1',
       'Ville_naissance_vendeur_1', 'Departement_naissance_vendeur_1',
       'Pays_naissance_vendeur_1', 'Date_naissance_vendeur_2',
       'Ville_naissance_vendeur_2', 'Departement_naissance_vendeur_2',
       'Pays_naissance_vendeur_2', 'Adresse_vendeur', 'Nom_acheteur_1',
       'Profession_acheteur_1', 'Date_naissance_acheteur_1',
       'Profession_acheteur_2', 'Date_naissance_acheteur_2',
       'Adresse_acheteur', 'Indexation_rente', 'Date_acte_authentique',
       'Payeur_Com', 'Chambre_de_bonnes', 'Nb_pieces', 'Etage',
       'Mettres_carres', 'Ascenseur', 'Parking_Box', 'Cave', 'Date_immeuble',
       'Pierre_de_taille', 'source', 'Ch_de_service', 'Parking', 'Bonus',
       'annee', 'type', 'quality', 'justenom1', 'societe',
       'vsociete', 'asociete', 'legasse', 'nblegassenet', 'reprise', 'limite',
       'stranger1', 'euro', 'franc', 'downp', 'annuity', 'VL', 'VO', 'done1']

colonnes2 = ['nom2', 'prenom2', 'annee2', 'mois2', 'jour2',
       'dep2', 'abroad2','lieutete2','sexe2',
       'Adresse_bien_vendu', 'Prenom_vendeur_1', 'Nom_vendeur_1',
       'Nom_vendeur_2', 'Nom_acheteur_2', 'Type_de_viager', 'Majoration',
       'Bouquet', 'Rente', 'Tete_1', 'Tete_2', 'Commission', 'Milliemes',
       'Standing', 'Remarques', 'Valeur_libre', 'Valeur_occupee', 'Numero',
       'Repertoire', 'Prenom_vendeur_2', 'Date_naissance_vendeur_1',
       'Ville_naissance_vendeur_1', 'Departement_naissance_vendeur_1',
       'Pays_naissance_vendeur_1', 'Date_naissance_vendeur_2',
       'Ville_naissance_vendeur_2', 'Departement_naissance_vendeur_2',
       'Pays_naissance_vendeur_2', 'Adresse_vendeur', 'Nom_acheteur_1',
       'Profession_acheteur_1', 'Date_naissance_acheteur_1',
       'Profession_acheteur_2', 'Date_naissance_acheteur_2',
       'Adresse_acheteur', 'Indexation_rente', 'Date_acte_authentique',
       'Payeur_Com', 'Chambre_de_bonnes', 'Nb_pieces', 'Etage',
       'Mettres_carres', 'Ascenseur', 'Parking_Box', 'Cave', 'Date_immeuble',
       'Pierre_de_taille', 'source', 'Ch_de_service', 'Parking', 'Bonus',
       'annee', 'type', 'quality', 'justenom1', 'societe',
       'vsociete', 'asociete', 'legasse', 'nblegassenet', 'reprise', 'limite',
       'stranger1', 'euro', 'franc', 'downp', 'annuity', 'VL', 'VO', 'done1']

df_travail = pd.DataFrame(index = list(range(1,df_grille['nom1'].count()+df_grille['nom2'].count()+1)),
                          columns = colonnes)

df1 = df_grille[pd.notnull(df_grille['nom1'])][colonnes1]
df2 = df_grille[pd.notnull(df_grille['nom2'])][colonnes2]

df1 = df1.rename(columns={'nom1': 'nom', 'prenom1' : 'prenoms',
                          'annee1' : 'ybirth' , 'mois1' : 'mbirth',
       'jour1' : 'dbirth', 'lieutete1' : 'villebirth',
                          'dep1' : 'Dpt_naissance',
       'abroad1':'abroad', 'sexe1' : 'sexe'})
df2 = df2.rename(columns={'nom2': 'nom', 'prenom2' : 'prenoms',
                          'annee2' : 'ybirth' , 'mois2' : 'mbirth',
       'jour2' : 'dbirth', 'lieutete2' : 'villebirth',
                          'dep2' : 'Dpt_naissance',
       'abroad2':'abroad', 'sexe2' : 'sexe'})


df_travail = pd.concat([df1, df2], ignore_index = True)
df_travail['Dpt_naissance'] = pd.to_numeric(df_travail['Dpt_naissance'], errors='coerce').astype('Int64').astype(str)
df_travail['sexe'] = df_travail['sexe'].astype(float)
df_travail['nom'] = df_travail['nom'].str.upper()

# colonne pour pouvoir identifier les individus (et non plus les paires de têtes)
df_travail['identification'] = list(range(1,df_travail.shape[0]+1))
df_travail = df_travail.append(df_grille[])

In [16]:
df_travail.to_csv('df_travail.csv')

In [24]:
df_grille

,Adresse_bien_vendu,Prenom_vendeur_1,Nom_vendeur_1,Nom_vendeur_2,Nom_acheteur_2,Type_de_viager,Majoration,Bouquet,Rente,Tete_1,...,reprise,limite,stranger1,euro,franc,downp,annuity,VL,VO,done1
0,"12 rue Raffet, 75016, Paris",Paule Marie,Abomes,NaN,NaN,Occupé,0-3 (35%) ; 3-6 (30%) ; 6-10 (25%) ; 10-99 (20%),690000 francs,120000 francs,Vendeur 1,...,0,0,0,0,0,105189.821890,18293.882068,NaN,NaN,1
1,"7 rue Louis David, 75016, Paris",Paule Marie,Abomes,NaN,NaN,Libre,NaN,340000 francs,84000 francs,Vendeur 1,...,0,0,0,0,0,51832.665861,12805.717448,NaN,NaN,1
2,"7 rue de Courcelles, 75008, Paris",Monika,Abrama,NaN,NaN,Occupé,NaN,25000 francs,48000 francs,Vendeur 1,...,0,0,1,0,0,3811.225431,7317.552827,NaN,NaN,1
3,"7 rue de Courcelles, 75008, Paris",Nicolas,Pinel,NaN,Baize,Occupé ; Libre (13),NaN,380000 francs,59640 francs,"Abrhama, Madame ; 21 mai 1925 ; Riga (Letonie)",...,1,1,1,0,1,57930.626550,9092.059388,213428.62413,152449.01724,1
4,"24 rue François 1er, 75008, Paris",Etienne,Adam,NaN,Jalabert,Occupé,0-99 (30%),2850000 francs,240000 francs,Vendeur 1,...,0,0,0,0,0,434479.699130,36587.764137,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,"107 rue Cardinet, 75017, Paris",Simone,de Nattes,NaN,NaN,Occupé,0-5 (30%) ; 5-10 (25%) ; 10-99 (20%),75000 francs,44400 francs,Vendeur 1,...,0,0,0,0,0,11433.676293,6768.736365,NaN,NaN,1
1046,"96 rue de Levis, 75017, Paris",Colette Louise,de Nattes,/,/,Occupé,0-5 (30%) ; 5-10 (25%) ; 10-99 (20%),200000 francs,72000 francs,Vendeur 1,...,0,0,0,0,0,30489.803447,10976.329241,NaN,NaN,1
1047,"54 rue de Boulainvilliers, 75016, Paris",Claude Antoinette Marie,de Plan de Sieyes de Veynes,/,/,Occupé,0-5 (30%) ; 5-10 (25%) ; 10-99 (20%),280000 francs,60000 francs,Vendeur 1,...,0,0,0,0,0,42685.724826,9146.941034,NaN,NaN,1
1048,"39 bis avenue Paul Doumer, 75116, Paris",William,Abib,Hanine,NaN,Occupé,0-99 (30%),1170000 francs,42493.44 francs,"de la Bigne, Madame ; 11 janvier 1911 ; Lorien...",...,1,0,0,0,0,178365.350170,6478.083167,NaN,NaN,1


## MERGE DATA FRAME CLIENT ET INSEE

In [ ]:
df_travail = pd.read_csv('df_travail.csv')
merged_df = df_travail.merge(df_insee1, on=['nom','sexe','ybirth','mbirth','dbirth','Dpt_naissance'], how='left')


merged_df = merged_df.dropna(subset = ['ydeath'] )

In [ ]:
merged_df['prenoms_x'] = merged_df['prenoms_x'].str.upper()
merged_df['prenoms_y'] = merged_df['prenoms_y'].str.upper()
liste_verif = []
for i in tqdm(merged_df.index.tolist()):

  liste_unidecode = []
  liste_prenoms = []

  for element in str(merged_df['prenoms_y'][i]).split(' '):
    liste_unidecode.append(unidecode(element).strip())
    liste_prenoms.append(element.strip())

  if len(str(merged_df['prenoms_x'][i]).split(' '))==1 and pd.notnull(merged_df['prenoms_x'][i]) :

    if merged_df['prenoms_x'][i].strip() in liste_prenoms:
      liste_verif.append(merged_df['identification'][i])
    elif unidecode(merged_df['prenoms_x'][i]) in liste_unidecode :
      liste_verif.append(merged_df['identification'][i])
    elif pd.notnull(merged_df['prenoms_y'][i]) and unidecode(merged_df['prenoms_x'][i]).strip() == unidecode(merged_df['prenoms_y'][i]):
      liste_verif.append(merged_df['identification'][i])
    else:
      pass

  elif pd.notnull(merged_df['prenoms_x'][i]) and len(merged_df['prenoms_x'][i].split(' '))>1:
    for element in merged_df['prenoms_x'][i].split(' '):
      if element in liste_prenoms or unidecode(element) in liste_unidecode:
        liste_verif.append(merged_df['identification'][i])
  else:
    pass

liste_verif = list(set(liste_verif))
print("Il y a",len(liste_verif),"individus pour qui les prénoms coïncident")

100%|██████████| 570/570 [00:00<00:00, 2900.21it/s]


Il y a 530 individus pour qui les prénoms coïncident


In [ ]:
df = merged_df[merged_df['identification'].isin(liste_verif)]
df_travail_bis = df_travail.drop(df_travail[df_travail['identification'].isin(liste_verif)].index)
merged_df_bis = df_travail_bis.merge(df_insee1, on=['nom','sexe','ybirth'], how='left')
#df.to_csv('2000-2022_match_parfait.csv')

In [ ]:
merged_df_bis['prenoms_x'] = merged_df_bis['prenoms_x'].str.upper()
merged_df_bis['prenoms_y'] = merged_df_bis['prenoms_y'].str.upper()
liste_verif = []
for i in tqdm(merged_df_bis.index.tolist()):

  liste_unidecode = []
  liste_prenoms = []

  for element in str(merged_df_bis['prenoms_y'][i]).split(' '):
    liste_unidecode.append(unidecode(element).strip())
    liste_prenoms.append(element.strip())

  if len(str(merged_df_bis['prenoms_x'][i]).split(' '))==1 and pd.notnull(merged_df_bis['prenoms_x'][i]) :

    if merged_df_bis['prenoms_x'][i].strip() in liste_prenoms:
      liste_verif.append(i)
    elif unidecode(merged_df_bis['prenoms_x'][i]) in liste_unidecode :
      liste_verif.append(i)
    elif pd.notnull(merged_df_bis['prenoms_y'][i]) and unidecode(merged_df_bis['prenoms_x'][i]).strip() == unidecode(merged_df_bis['prenoms_y'][i]):
      liste_verif.append(i)
    else:
      pass

  elif pd.notnull(merged_df_bis['prenoms_x'][i]) and len(merged_df_bis['prenoms_x'][i].split(' '))>1:
    for element in merged_df_bis['prenoms_x'][i].split(' '):
      if element in liste_prenoms or unidecode(element) in liste_unidecode:
        liste_verif.append(i)
  else:
    pass

liste_verif = list(set(liste_verif))
print("Il y a",len(liste_verif),"individus pour qui les prénoms coïncident")

100%|██████████| 10787/10787 [00:03<00:00, 3253.04it/s]

Il y a 1049 individus pour qui les prénoms coïncident


In [ ]:
df2 = merged_df_bis[merged_df_bis.index.isin(liste_verif)]
#df2.to_csv('1975-2000_match_imparfaits.csv')

In [ ]:
df1 = pd.read_csv('1975-2000_match_parfait.csv')
df2 = pd.read_csv('2000-2022_match_parfait.csv')
df_parfait = pd.concat([df1,df2],ignore_index = True)
df_parfait.to_csv('df_parfait.csv')

df3 = pd.read_csv('1975-2000_match_imparfaits.csv')
df4 = pd.read_csv('2000-2022_match_imparfaits.csv')
df_imparfait = pd.concat([df3,df4],ignore_index = True)
df_imparfait.to_csv('df_imparfait.csv')

# Phase 2 : Nettoyage de la base de travail

## Gestion du DATA FRAME IMPARFAIT


### Traitement 1

In [ ]:
df = pd.read_csv('df_imparfait.csv')
df = df.drop(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'], axis = 1)
df['Dpt_naissance_y'] = df['Dpt_naissance_y'].astype(float)
df['Dpt_naissance_x'] = df['Dpt_naissance_x'].astype(float)
keep_dpt_dpt = df[df['Dpt_naissance_y']==df['Dpt_naissance_x']].index.to_list()
df_keep_dpt = df[df['Dpt_naissance_y']==df['Dpt_naissance_x']]
df = df.drop(keep_dpt_dpt)

In [ ]:
for i in tqdm(df_keep.index.tolist()):
  for j in df.index.tolist():
    if (df['nom'][j] == df_keep['nom'][i] and df['ybirth'][j] == df_keep['ybirth'][i] and df['sexe'][j] == df_keep['sexe'][i]):
      df = df.drop(j)
df

In [ ]:
# Méthode de Levenshtein sur les prénoms
df['prenoms_x'] = df['prenoms_x'].str.upper()
keep_prenoms = []
for i in df.index.tolist():
  if len(df['prenoms_x'][i].split(' '))==1 and len(df['prenoms_y'][i].split(' '))<=2:
    if Levenshtein.distance(df['prenoms_x'][i], df['prenoms_y'][i])<=10:
      keep_prenoms.append(i)
    else:
      pass
  if len(df['prenoms_x'][i].split(' '))==2 and len(df['prenoms_y'][i].split(' '))<=3:
    if Levenshtein.distance(df['prenoms_x'][i], df['prenoms_y'][i])<=10:
      keep_prenoms.append(i)
    else:
      pass
df_keep_prenoms = df[df.index.isin(keep_prenoms)]
df = df.drop(keep_prenoms)

In [ ]:
# Méthode de Levenshtein sur les villes de naissance
sub_df = df[pd.notnull(df['villebirth_x'])]
sub_df['villebirth_x'] = sub_df['villebirth_x'].str.upper()
keep_ville = []
for i in sub_df.index.tolist():
  if Levenshtein.distance(sub_df['villebirth_x'][i], sub_df['villebirth_y'][i])<=5:
    keep_ville.append(i)
  else:
    pass


<ipython-input-63-e90e261367b3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['villebirth_x'] = sub_df['villebirth_x'].str.upper()


In [ ]:
df = pd.concat([df_keep_dpt, df_keep_prenoms], ignore_index = True)
df.to_csv('fichier_imparfait_traitement1.csv')

In [ ]:
impossible = df[df['annee']> df['ydeath']+1][['annee','ydeath']].index.tolist()
df = df.drop(impossible)
df.to_csv('fichier_imparfait_traitement1.csv')

### Traitement 2

In [ ]:
df_parfait = pd.read_csv('df_parfait.csv')

In [ ]:
indices = df_parfait[df_parfait['annee']>df_parfait['ydeath']+1][['prenoms_x','prenoms_y','annee','ydeath']].index.to_list()
df_parfait = df_parfait.drop(indices)
df_parfait.to_csv('df_parfait.csv')

In [ ]:
df = pd.read_csv('fichier_imparfait_traitement1.csv')

In [ ]:
df[['nom', 'mbirth_x','mbirth_y','ybirth','dbirth_x','dbirth_y','prenoms_x','prenoms_y', 'Dpt_naissance_x','Dpt_naissance_y','Date_acte_authentique']][400:450]

,nom,mbirth_x,mbirth_y,ybirth,dbirth_x,dbirth_y,prenoms_x,prenoms_y,Dpt_naissance_x,Dpt_naissance_y,Date_acte_authentique
400,COLIN,12.0,7.0,1922.0,24.0,15.0,ELISE MARIE,RENE JEANNE MARIE,22.0,63.0,19DEC1989
401,COLIN,12.0,4.0,1922.0,24.0,8.0,ELISE MARIE,MARIE JOSEPH,22.0,56.0,19DEC1989
402,COLIN,12.0,5.0,1922.0,24.0,25.0,ELISE MARIE,LUCIENNE MARIE,22.0,54.0,19DEC1989
403,COLIN,12.0,2.0,1922.0,24.0,1.0,ELISE MARIE,BLANCHE MARIE,22.0,88.0,19DEC1989
404,COLIN,12.0,2.0,1922.0,24.0,24.0,ELISE MARIE,LOUISE MARIE LEONIE,22.0,50.0,19DEC1989
405,COLIN,12.0,4.0,1922.0,24.0,5.0,ELISE MARIE,MARIE GABRIELLE,22.0,88.0,19DEC1989
406,COLIN,12.0,2.0,1922.0,24.0,24.0,ELISE MARIE,ELIANE MARIE,22.0,59.0,19DEC1989
407,COLIN,12.0,11.0,1922.0,24.0,3.0,ELISE MARIE,GISELE LUCIE MARIE,22.0,75.0,19DEC1989
408,COLIN,12.0,9.0,1922.0,24.0,20.0,ELISE MARIE,HENRIETTE ELISE,22.0,38.0,19DEC1989
409,COLIN,12.0,2.0,1922.0,24.0,26.0,ELISE MARIE,YVETTE MARIE,22.0,54.0,19DEC1989


In [ ]:
liste_drop = [1,2,4,5,10,11,12,13,14,15,16,18,19,20,21,22,24,25,26, 28, 34,35,36,37,39,41,42,46,47,49, 50, 51, 54,58,60,61,62,63,64,68,70,72,73,74,76,78,82,86,87,88,89,91,92,97,98,100,101,103,104,105,106,107,108,109,110,111,113,114, 115, 116,117,118,121,123,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,141,144,145,146,148, 150,151,152,154,155,156,157,159,160,161,162,163,164,165,168,169,172,174,175,177,180,181,182,183,184,185,186,187,188,189,191,193,194,196,199, 200,201,202,203,204,208,218,219,220,221,222,223,224,227,228,229,232,233,234,235,238,239,240,244,249, 250,261,263,265,266,267,268,269,270,271,273,274,275,278,284,289,290,291,292,294,295,297,298,299,301,305,306,307,308,309,310,311,312,315,316,325,326,327,328,329,330,331,332,333,334,336,337,339,340,341,342,343,344,345,346,347,348,350,351,354,360,365,366,367,368,372,376,377,378,379,380,381,382,383,384,386,389,390,391,392,393,394,395,397,398,399,400,401,402,403,404,405,407,408,409,410,413]
#df = df.drop(l_drop)
df.to_csv('df_imparfait_post_tri_main.csv')

In [ ]:
l_drop = [145,146,147,148,90,105,120,60, 62,63,65,66, 97,96,95,94,93,155,137,136]
df_imparfait = df_imparfait.drop(l_drop)
df_imparfait.to_csv('df_imparfait_post_tri_main.csv')

### Merge parfait

In [ ]:
df_imparfait = pd.read_csv('df_imparfait_post_tri_main.csv')
df_parfait = pd.read_csv('df_parfait.csv')
df_imparfait = df_imparfait.sort_values(by = 'nom')

In [ ]:
liste_parfaite = [0, 1,2,3,8,9,14,15,17,18,19,20,21,22,23,24,26,29,31,33,34,37,39,40,41,42,43,44,46,47,49, 53,54,55,57,58,59,60,61,63,68,69,72,85,84,87,93,95,98,99,100,103,104,105,106,107,108,109,112,114,117,118,119,122,124,125,127,130,131,132,133,134,135,138,140,141]

In [ ]:
df_parfait = df_parfait.drop(['Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1', 'Unnamed: 0'], axis = 1)
df2 = df_imparfait[df_imparfait.index.isin(liste_parfaite)]
df2 = df2.drop(['Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1', 'Unnamed: 0'], axis = 1)
df = pd.concat([df_parfait,df2], ignore_index=True)
df_imparfait = df_imparfait.drop(liste_parfaite)
df_imparfait.to_csv('df_imparfait_post_merge.csv')
df.to_csv('merge_parfait.csv')
df

### Vérification que le Data Frame merge parfait convient


In [ ]:
df = pd.read_csv('merge_parfait.csv')
#df = df.drop([26,62,67,87,105,137,250,270,276,379,395,400,601,605,658,662,747])
df = df.drop([67,87,137])
df.columns
df = df[['nom', 'sexe','prenoms_x', 'prenoms_y', 'ybirth',
       'ydeath', 'mdeath', 'ddeath','paysbirth', 'mbirth', 'dbirth', 'mbirth_x', 'dbirth_x',
       'Dpt_naissance_x', 'mbirth_y', 'dbirth_y',
       'Dpt_naissance', 'abroad', 'villebirth_x', 'villebirth_y',  'Adresse_bien_vendu',
       'Prenom_vendeur_1', 'Nom_vendeur_1', 'Nom_vendeur_2', 'Nom_acheteur_2',
       'Type_de_viager', 'Majoration', 'Bouquet', 'Rente', 'Tete_1', 'Tete_2',
       'Commission', 'Milliemes', 'Standing', 'Remarques', 'Valeur_libre',
       'Valeur_occupee', 'Numero', 'Repertoire', 'Prenom_vendeur_2',
       'Date_naissance_vendeur_1', 'Ville_naissance_vendeur_1',
       'Departement_naissance_vendeur_1', 'Pays_naissance_vendeur_1',
       'Date_naissance_vendeur_2', 'Ville_naissance_vendeur_2',
       'Departement_naissance_vendeur_2', 'Pays_naissance_vendeur_2',
       'Adresse_vendeur', 'Nom_acheteur_1', 'Profession_acheteur_1',
       'Date_naissance_acheteur_1', 'Profession_acheteur_2',
       'Date_naissance_acheteur_2', 'Adresse_acheteur', 'Indexation_rente',
       'Date_acte_authentique', 'Payeur_Com', 'Chambre_de_bonnes', 'Nb_pieces',
       'Etage', 'Mettres_carres', 'Ascenseur', 'Parking_Box', 'Cave',
       'Date_immeuble', 'Pierre_de_taille', 'source', 'Ch_de_service',
       'Parking', 'Bonus', 'annee', 'type_x', 'quality', 'justenom1',
       'societe', 'vsociete', 'asociete', 'legasse', 'nblegassenet', 'reprise',
       'limite', 'stranger1', 'euro', 'franc', 'downp', 'annuity', 'VL', 'VO',
       'done1', 'identification',  'codebirth', 'codebirth.1', 'nbdeath',
       'type_y', 'reserve', 'fichier', 'allobs', 'Dpt_naissance_y']]

In [ ]:
df[df.duplicated( keep = False)].sort_values(by = 'nom')[['nom', 'sexe','prenoms_x', 'prenoms_y', 'ybirth',
       'ydeath','Adresse_bien_vendu', 'Nom_acheteur_1']]

,nom,sexe,prenoms_x,prenoms_y,ybirth,ydeath,Adresse_bien_vendu,Nom_acheteur_1


In [ ]:
df.to_csv('fichier_final_parfait.csv', index = False)

### Modification du fichier df_travail


In [71]:
df_travail = pd.read_csv('Merge_4/df_travail.csv')
df = pd.read_csv('Merge_4/df_parfait.csv')
d = pd.read_csv('final.csv')
df_travail['nom'] = df_travail['nom'].str.upper()
df_travail['prenoms'] = df_travail['prenoms'].str.upper()
df_travail['match'] = 0
df_travail['head'] = 0
df_travail['head'][:1049] = 1
df_travail['head'][1049:]=2

# Match 1 et 2
for i in tqdm(df.index.tolist()):
  for j in df_travail.index.tolist():
    if df['nom'][i]==df_travail['nom'][j] and df['prenoms_x'][i]==df_travail['prenoms'][j] :
      df_travail['match'][j] =1

for l in tqdm(d.index.tolist()):
  for j in df_travail.index.tolist():
    if d['nom'][l]==df_travail['nom'][j] and d['prenoms_x'][l]==df_travail['prenoms'][j] :
      if df_travail['match'][j]==0:
        df_travail['match'][j] = 2


<ipython-input-71-2efaad7c1e5c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_travail['head'][:1049] = 1
<ipython-input-71-2efaad7c1e5c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_travail['head'][1049:]=2
  0%|          | 0/826 [00:00<?, ?it/s]<ipython-input-71-2efaad7c1e5c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_travail['match'][j] =1
 91%|█████████ | 812/890 [00:09<00:01, 77.79it/s]<ipython-input-71-2efaa

In [72]:
df_travail[df_travail['match']==2]

,Unnamed: 0,nom,prenoms,ybirth,mbirth,dbirth,Dpt_naissance,abroad,villebirth,sexe,...,euro,franc,downp,annuity,VL,VO,done1,identification,match,head
17,17,ANCIER,CHARLES CHALOUM,1906.0,7.0,18.0,99.0,1.0,NaN,1.0,...,0,1,125008.194130,4024.654055,167693.918960,91469.410342,1,18,2,1
116,116,BITNER,NADINE FABIENNE,1932.0,11.0,13.0,92.0,0.0,NaN,2.0,...,0,0,99086.000000,3660.000000,NaN,NaN,1,117,2,1
144,144,BOULDOIRE,JEANNE SUZANNE CÉCILE,1918.0,2.0,9.0,75.0,0.0,NaN,2.0,...,0,0,45734.705171,5488.164620,NaN,NaN,1,145,2,1
162,162,BREUILLARD,EDMOND,1909.0,11.0,20.0,93.0,0.0,NaN,1.0,...,0,0,12958.166465,8872.532803,NaN,NaN,1,163,2,1
163,163,BREUILLARD,EDMOND,1909.0,11.0,20.0,93.0,0.0,NaN,1.0,...,0,0,4573.470517,9329.879855,91469.410342,NaN,1,164,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290,1290,BOSSAN,MARCELLE,1911.0,2.0,25.0,NaN,NaN,NaN,2.0,...,0,0,7622.450862,2561.143490,NaN,NaN,0,1291,2,2
1291,1291,BOSSAN,MARCELLE,1911.0,2.0,25.0,NaN,NaN,NaN,2.0,...,0,0,17531.636982,10976.329241,NaN,NaN,0,1292,2,2
1311,1311,LEROY,ANDRÉE YVONNE,1921.0,6.0,10.0,45.0,0.0,NaN,2.0,...,0,0,26678.578017,3475.837593,72413.283188,NaN,1,1312,2,2
1316,1316,LABOUREUR,DENISE CÉCILE MARTHE,1923.0,6.0,25.0,76.0,0.0,NaN,2.0,...,0,0,9146.941034,6951.675186,NaN,NaN,1,1317,2,2


In [74]:
df_travail.to_csv('travail_final.csv')

In [93]:
m = df_travail[df_travail['match']==2]
m[m['Type_de_viager']== 'Libre']

,Unnamed: 0,nom,prenoms,ybirth,mbirth,dbirth,Dpt_naissance,abroad,villebirth,sexe,...,euro,franc,downp,annuity,VL,VO,done1,identification,match,head
163,163,BREUILLARD,EDMOND,1909.0,11.0,20.0,93.0,0.0,NaN,1.0,...,0,0,4573.470517,9329.879855,91469.410342,NaN,1,164,2,1
171,171,BROSSARD,ODETTE,1927.0,9.0,19.0,NaN,NaN,NaN,2.0,...,0,0,53357.156033,2196.430559,NaN,NaN,0,172,2,1
195,195,CATTALAN,FRANÇOIS,1912.0,3.0,3.0,75.0,0.0,NaN,1.0,...,0,0,27440.823103,5122.286979,NaN,NaN,1,196,2,1
281,281,DEFERT,PIERRE ALCIDE,1922.0,1.0,13.0,NaN,NaN,NaN,1.0,...,0,0,19056.127155,4024.654055,NaN,NaN,0,282,2,1
289,289,DELMAS,ROBERT,1911.0,7.0,23.0,NaN,NaN,NaN,1.0,...,0,0,17684.086000,2232.402429,39636.744482,NaN,0,290,2,1
296,296,DERI,EDITH,1921.0,3.0,19.0,99.0,1.0,NaN,2.0,...,0,0,17000.000000,36000.000000,NaN,NaN,1,297,2,1
324,324,DUBOIS,RENÉ ANDRÉ,1927.0,4.0,3.0,80.0,0.0,NaN,1.0,...,0,0,5335.715603,13720.411551,NaN,NaN,1,325,2,1
326,326,DUCHEZ,MARIE ELISABETH,1923.0,6.0,6.0,17.0,0.0,Jonzac (17),2.0,...,0,0,57931.000000,37797.600000,NaN,NaN,1,327,2,1
359,359,ETIENNE,SUZANNE MARIE,1917.0,8.0,17.0,NaN,NaN,NaN,2.0,...,0,0,27440.823103,5488.164620,83846.959481,NaN,0,360,2,1
393,393,FREY,RAYMOND EUGÈNE,1913.0,11.0,27.0,NaN,NaN,NaN,1.0,...,0,0,33538.783792,2539.190831,NaN,NaN,0,394,2,1


In [13]:
df = pd.read_csv('data_agence.csv', encoding='ISO-8859-1')

In [10]:
df['Tete_1'].count()

1049

# Phase 3 : Derniers nettoyages

## Création de la base

In [7]:
df_grille = pd.read_csv('data_agence.csv', encoding='ISO-8859-1')

def convert_to_timestamp(value):
    if pd.isnull(value) or value == '/':
        return value
    else:
        try:
            timestamp = dateparser.parse(str(value))
            return timestamp
        except ValueError:
            return value

df_grille['Date_naissance_vendeur_1'] = df_grille['Date_naissance_vendeur_1'].apply(convert_to_timestamp)
df_grille['Date_naissance_vendeur_2'] = df_grille['Date_naissance_vendeur_2'].apply(convert_to_timestamp)
df_grille = df_grille.dropna(subset = 'Tete_1')
df_grille['Tete_1'] = df_grille['Tete_1'].astype(str)
df_grille['Tete_2'] = df_grille['Tete_2'].astype(str)



colonnes = ['Adresse_bien_vendu', 'Prenom_vendeur_1', 'Nom_vendeur_1',
       'Nom_vendeur_2', 'Nom_acheteur_2', 'Type_de_viager', 'Majoration',
       'Bouquet', 'Rente', 'Tete_1', 'Tete_2', 'Commission', 'Milliemes',
       'Standing', 'Remarques', 'Valeur_libre', 'Valeur_occupee', 'Numero',
       'Repertoire', 'Prenom_vendeur_2', 'Date_naissance_vendeur_1',
       'Ville_naissance_vendeur_1', 'Departement_naissance_vendeur_1',
       'Pays_naissance_vendeur_1', 'Date_naissance_vendeur_2',
       'Ville_naissance_vendeur_2', 'Departement_naissance_vendeur_2',
       'Pays_naissance_vendeur_2', 'Adresse_vendeur', 'Nom_acheteur_1',
       'Profession_acheteur_1', 'Date_naissance_acheteur_1',
       'Profession_acheteur_2', 'Date_naissance_acheteur_2',
       'Adresse_acheteur', 'Indexation_rente', 'Date_acte_authentique',
       'Payeur_Com', 'Chambre_de_bonnes', 'Nb_pieces', 'Etage',
       'Mettres_carres', 'Ascenseur', 'Parking_Box', 'Cave', 'Date_immeuble',
       'Pierre_de_taille', 'source', 'Ch_de_service', 'Parking', 'Bonus',
       'annee', 'type', 'nom1', 'nom2', 'prenom1', 'prenom2', 'sexe1', 'sexe2',
       'annee1', 'annee2', 'naissancetete1', 'mois1', 'jour1',
       'naissancetete2', 'mois2', 'jour2', 'lieutete1', 'dep1', 'lieutete2',
       'dep2', 'abroad1', 'abroad2', 'quality', 'nom', 'justenom1', 'societe',
       'vsociete', 'asociete', 'legasse', 'nblegassenet', 'reprise', 'limite',
       'stranger1', 'euro', 'franc', 'downp', 'annuity', 'VL', 'VO', 'done1',
       'ville1', 'ville2']

colonnes1 = ['nom1', 'prenom1', 'annee1', 'mois1', 'jour1',
       'dep1', 'abroad1','ville1','sexe1',
        'Adresse_bien_vendu', 'Prenom_vendeur_1', 'Nom_vendeur_1',
       'Nom_vendeur_2', 'Nom_acheteur_2', 'Type_de_viager', 'Majoration',
       'Bouquet', 'Rente', 'Tete_1', 'Tete_2', 'Commission', 'Milliemes',
       'Standing', 'Remarques', 'Valeur_libre', 'Valeur_occupee', 'Numero',
       'Repertoire', 'Prenom_vendeur_2', 'Date_naissance_vendeur_1',
       'Ville_naissance_vendeur_1', 'Departement_naissance_vendeur_1',
       'Pays_naissance_vendeur_1', 'Date_naissance_vendeur_2',
       'Ville_naissance_vendeur_2', 'Departement_naissance_vendeur_2',
       'Pays_naissance_vendeur_2', 'Adresse_vendeur', 'Nom_acheteur_1',
       'Profession_acheteur_1', 'Date_naissance_acheteur_1',
       'Profession_acheteur_2', 'Date_naissance_acheteur_2',
       'Adresse_acheteur', 'Indexation_rente', 'Date_acte_authentique',
       'Payeur_Com', 'Chambre_de_bonnes', 'Nb_pieces', 'Etage',
       'Mettres_carres', 'Ascenseur', 'Parking_Box', 'Cave', 'Date_immeuble',
       'Pierre_de_taille', 'source', 'Ch_de_service', 'Parking', 'Bonus',
       'annee', 'type', 'quality', 'justenom1', 'societe',
       'vsociete', 'asociete', 'legasse', 'nblegassenet', 'reprise', 'limite',
       'stranger1', 'euro', 'franc', 'downp', 'annuity', 'VL', 'VO', 'done1']

colonnes2 = ['nom2', 'prenom2', 'annee2', 'mois2', 'jour2',
       'dep2', 'abroad2','ville2','sexe2',
       'Adresse_bien_vendu', 'Prenom_vendeur_1', 'Nom_vendeur_1',
       'Nom_vendeur_2', 'Nom_acheteur_2', 'Type_de_viager', 'Majoration',
       'Bouquet', 'Rente', 'Tete_1', 'Tete_2', 'Commission', 'Milliemes',
       'Standing', 'Remarques', 'Valeur_libre', 'Valeur_occupee', 'Numero',
       'Repertoire', 'Prenom_vendeur_2', 'Date_naissance_vendeur_1',
       'Ville_naissance_vendeur_1', 'Departement_naissance_vendeur_1',
       'Pays_naissance_vendeur_1', 'Date_naissance_vendeur_2',
       'Ville_naissance_vendeur_2', 'Departement_naissance_vendeur_2',
       'Pays_naissance_vendeur_2', 'Adresse_vendeur', 'Nom_acheteur_1',
       'Profession_acheteur_1', 'Date_naissance_acheteur_1',
       'Profession_acheteur_2', 'Date_naissance_acheteur_2',
       'Adresse_acheteur', 'Indexation_rente', 'Date_acte_authentique',
       'Payeur_Com', 'Chambre_de_bonnes', 'Nb_pieces', 'Etage',
       'Mettres_carres', 'Ascenseur', 'Parking_Box', 'Cave', 'Date_immeuble',
       'Pierre_de_taille', 'source', 'Ch_de_service', 'Parking', 'Bonus',
       'annee', 'type', 'quality', 'justenom1', 'societe',
       'vsociete', 'asociete', 'legasse', 'nblegassenet', 'reprise', 'limite',
       'stranger1', 'euro', 'franc', 'downp', 'annuity', 'VL', 'VO', 'done1']

df_travail = pd.DataFrame(index = list(range(1,df_grille['nom1'].count()+df_grille['nom2'].count()+1)),
                          columns = colonnes)

df1 = df_grille[pd.notnull(df_grille['nom1'])][colonnes1]
df2 = df_grille[pd.notnull(df_grille['nom2'])][colonnes2]

df1 = df1.rename(columns={'nom1': 'A_nom', 'prenom1' : 'A_prenoms',
                          'annee1' : 'A_annee' , 'mois1' : 'A_mois',
       'jour1' : 'A_jour', 'ville1' : 'A_ville',
                          'dep1' : 'A_departement',
       'abroad1':'A_abroad', 'sexe1' : 'A_sexe'})
df2 = df2.rename(columns={'nom2': 'A_nom', 'prenom2' : 'A_prenoms',
                          'annee2' : 'A_annee' , 'mois2' : 'A_mois',
       'jour2' : 'A_jour', 'ville2' : 'A_ville',
                          'dep2' : 'A_departement',
       'abroad2':'A_abroad', 'sexe2' : 'A_sexe'})

df1['head'] = 1
df2['head'] = 2
df_travail = pd.concat([df1, df2], ignore_index = True)
df_travail['A-departement'] = pd.to_numeric(df_travail['A_departement'], errors='coerce').astype('Int64').astype(str)
df_travail['A_sexe'] = df_travail['A_sexe'].astype(float)
df_travail['A_nom'] = df_travail['A_nom'].str.upper()

# création des colonnes de match avec le df INSEE
df_travail['nom'] = df_travail['A_nom']
df_travail['sexe'] = df_travail['A_sexe']
df_travail['ybirth'] = df_travail['A_annee']
df_travail['mbirth'] = df_travail['A_mois']
df_travail['dbirth'] = df_travail['A_jour']
df_travail['dpt_naissance'] = df_travail['A_departement']

#df_travail.to_csv('final.csv')

In [9]:
df_parfait = pd.read_csv('Merge_4/merge_parfait.csv')
df_imparfait = df_parfait[826:]
df_parfait = df_parfait[:826]

In [10]:
# Gestion du Data Frame parfait

df_parfait['B_prenoms'] = df_parfait['prenoms_y']
df_parfait['B_ville'] = df_parfait['villebirth_y']
df_parfait['B_pays'] = df_parfait['paysbirth']
df_parfait['B_ville'] = df_parfait['villebirth_y']
df_parfait['B_nom'] = df_parfait['nom']
df_parfait['B_sexe'] = df_parfait['sexe']
df_parfait['type'] = df_parfait['type_y']
df_parfait['B_jour'] = df_parfait['dbirth']
df_parfait['B_mois'] = df_parfait['mbirth']
df_parfait['B_annee'] = df_parfait['ybirth']
df_parfait['D_jour'] = df_parfait['ddeath']
df_parfait['D_mois'] = df_parfait['mdeath']
df_parfait['D_annee'] = df_parfait['ydeath']
df_parfait['dpt_naissance'] = df_parfait['Dpt_naissance']
df_parfait['B_departement'] = df_parfait['Dpt_naissance']
df_parfait['match'] = 1


df_parfait = df_parfait[['nom', 'sexe', 'ybirth', 'mbirth','dbirth','dpt_naissance', 'B_sexe', 'B_nom','B_prenoms', 'B_mois', 'B_jour', 'B_annee'
 ,'B_departement','D_annee', 'D_mois', 'D_jour', 'B_pays', 'B_ville', 'abroad', 'codebirth', 'nbdeath', 'reserve',
       'fichier', 'allobs', 'match']]

# Gestion du Data Frame imparfait

df_imparfait['B_prenoms'] = df_imparfait['prenoms_y']
df_imparfait['B_ville'] = df_imparfait['villebirth_y']
df_imparfait['B_pays'] = df_imparfait['paysbirth']
df_imparfait['B_ville'] = df_imparfait['villebirth_y']
df_imparfait['B_nom'] = df_imparfait['nom']
df_imparfait['B_sexe'] = df_imparfait['sexe']
df_imparfait['type'] = df_imparfait['type_y']
df_imparfait['B_jour'] = df_imparfait['dbirth_y']
df_imparfait['B_mois'] = df_imparfait['mbirth_y']
df_imparfait['B_annee'] = df_imparfait['ybirth']
df_imparfait['D_jour'] = df_imparfait['ddeath']
df_imparfait['D_mois'] = df_imparfait['mdeath']
df_imparfait['D_annee'] = df_imparfait['ydeath']
df_imparfait['dpt_naissance'] = df_imparfait['Dpt_naissance']
df_imparfait['B_departement'] = df_imparfait['Dpt_naissance_y']
df_imparfait['match'] = 2




df_imparfait = df_imparfait[['nom', 'sexe', 'ybirth', 'mbirth','dbirth','dpt_naissance', 'B_sexe', 'B_nom','B_prenoms', 'B_mois', 'B_jour', 'B_annee'
 ,'B_departement','D_annee', 'D_mois', 'D_jour', 'B_pays', 'B_ville', 'abroad', 'codebirth', 'nbdeath', 'reserve',
       'fichier', 'allobs', 'match']]



# Merges
df_tomerge = df_parfait.append(df_imparfait)
merged_df = pd.merge(df_travail, df_tomerge, on=['nom', 'sexe', 'ybirth', 'mbirth','dbirth','dpt_naissance'], how = 'left')
merged_df = merged_df.drop_duplicates()
#merged_df.to_csv('df.csv')

<ipython-input-10-d385961b63c3>:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_tomerge = df_parfait.append(df_imparfait)


## Agencement des colonnes

In [186]:
df = pd.read_csv('df.csv')

In [192]:
df = df[['A_nom',
 'A_prenoms',
 'A_annee',
 'A_mois',
 'A_jour',
 'A_departement',
 'A_abroad',
 'A_ville',
 'A_sexe',
 'A-departement',
 'B_nom',
 'B_sexe',
 'B_prenoms',
 'B_mois',
 'B_jour',
 'B_annee',
 'B_pays',
 'B_ville',
 'B_departement',
 'D_annee',
 'D_mois',
 'D_jour',
  'head',
  'match',
 'abroad',
 'codebirth',
 'nbdeath',
 'reserve',
 'fichier',
 'allobs',
 'Adresse_bien_vendu',
 'Prenom_vendeur_1',
 'Nom_vendeur_1',
 'Nom_vendeur_2',
 'Nom_acheteur_2',
 'Type_de_viager',
 'Majoration',
 'Bouquet',
 'Rente',
 'Tete_1',
 'Tete_2',
 'Commission',
 'Milliemes',
 'Standing',
 'Remarques',
 'Valeur_libre',
 'Valeur_occupee',
 'Numero',
 'Repertoire',
 'Prenom_vendeur_2',
 'Date_naissance_vendeur_1',
 'Ville_naissance_vendeur_1',
 'Departement_naissance_vendeur_1',
 'Pays_naissance_vendeur_1',
 'Date_naissance_vendeur_2',
 'Ville_naissance_vendeur_2',
 'Departement_naissance_vendeur_2',
 'Pays_naissance_vendeur_2',
 'Adresse_vendeur',
 'Nom_acheteur_1',
 'Profession_acheteur_1',
 'Date_naissance_acheteur_1',
 'Profession_acheteur_2',
 'Date_naissance_acheteur_2',
 'Adresse_acheteur',
 'Indexation_rente',
 'Date_acte_authentique',
 'Payeur_Com',
 'Chambre_de_bonnes',
 'Nb_pieces',
 'Etage',
 'Mettres_carres',
 'Ascenseur',
 'Parking_Box',
 'Cave',
 'Date_immeuble',
 'Pierre_de_taille',
 'source',
 'Ch_de_service',
 'Parking',
 'Bonus',
 'annee',
 'type',
 'quality',
 'justenom1',
 'societe',
 'vsociete',
 'asociete',
 'legasse',
 'nblegassenet',
 'reprise',
 'limite',
 'stranger1',
 'euro',
 'franc',
 'downp',
 'annuity',
 'VL',
 'VO',
 'done1',
 'nom',
 'sexe',
 'ybirth',
 'mbirth',
 'dbirth',
 'dpt_naissance']]

## Chercher l'individu manquant

In [197]:
df['B_nom'][30] = 'AUBERT'
df['B_sexe'][30] = 'NICOLLE'
df['B_prenoms'][30] = float(2)
df['B_mois'][30] = float(12)
df['B_jour'][30] = float(16)
df['B_annee'][30] = float(1924)
df['B_pays'][30] = np.nan
df['B_departement'][30] = float(54)
df['B_ville'][30] = 'NEUVES_MAISONS'
df['D_annee'][30] = float(2011)
df['D_mois'][30] = float(1)
df['D_jour'][30] = float(17)
df['codebirth'][30] = '54397'
df['nbdeath'][30] = 'N'
df['reserve'][30] = np.nan
df['fichier'][30] = float(2011)
df['allobs'][30] = "6135550    b'AUBERT*NICOLLE/ "
df['match'][30] = 4
df['head'][30] = 1

<ipython-input-197-823c5bab5f38>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['B_nom'][30] = 'AUBERT'
<ipython-input-197-823c5bab5f38>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['B_prenoms'][30] = float(2)
<ipython-input-197-823c5bab5f38>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['B_mois'][30] = float(12)
<ipython-input-197-823c5bab5f38>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

In [4]:
df = pd.read_csv('travail_final.csv')
df['D_annee'].count()

827

In [30]:
df[df.duplicated(subset= ['A_nom', 'A_prenoms', 'A_annee', 'A_mois', 'A_jour', 'A_departement',
       'A_abroad', 'A_ville', 'A_sexe', 'A-departement', 'Adresse_bien_vendu',
 'Prenom_vendeur_1',
 'Nom_vendeur_1',
 'Nom_vendeur_2',
 'Nom_acheteur_2',
 'Type_de_viager',
 'Majoration',
 'Bouquet',
 'Rente',
 'Tete_1',
 'Tete_2',
 'Commission',
 'Milliemes',
 'Standing',
 'Remarques',
 'Valeur_libre',
 'Valeur_occupee',
 'Numero',
 'Repertoire',
 'Prenom_vendeur_2',
 'Date_naissance_vendeur_1',
 'Ville_naissance_vendeur_1',
 'Departement_naissance_vendeur_1',
 'Pays_naissance_vendeur_1',
 'Date_naissance_vendeur_2',
 'Ville_naissance_vendeur_2',
 'Departement_naissance_vendeur_2',
 'Pays_naissance_vendeur_2',
 'Adresse_vendeur',
 'Nom_acheteur_1',
 'Profession_acheteur_1',
 'Date_naissance_acheteur_1',
 'Profession_acheteur_2',
 'Date_naissance_acheteur_2',
 'Adresse_acheteur',
 'Indexation_rente',
 'Date_acte_authentique',
 'Payeur_Com',
 'Chambre_de_bonnes',
 'Nb_pieces',
 'Etage',
 'Mettres_carres',
 'Ascenseur',
 'Parking_Box',
 'Cave',
 'Date_immeuble',
 'Pierre_de_taille',
 'source',
 'Ch_de_service',
 'Parking',
 'Bonus',
 'annee',
 'type',
 'quality',
 'justenom1',
 'societe',
 'vsociete',
 'asociete',
 'legasse',
 'nblegassenet',
 'reprise',
 'limite'], keep=False)]

,A_nom,A_prenoms,A_annee,A_mois,A_jour,A_departement,A_abroad,A_ville,A_sexe,A-departement,...,annuity,VL,VO,done1,nom,sexe,ybirth,mbirth,dbirth,dpt_naissance
241,COLLAS DE CHATELPERRON,Marie Elisabeth Louis,1924.0,9.0,12.0,3.0,0.0,Moulins,2.0,3.0,...,4756.409338,NaN,NaN,1,COLLAS DE CHATELPERRON,2.0,1924.0,9.0,12.0,3.0
242,COLLAS DE CHATELPERRON,Marie Elisabeth Louis,1924.0,9.0,12.0,3.0,0.0,Moulins,2.0,3.0,...,4756.409338,NaN,NaN,1,COLLAS DE CHATELPERRON,2.0,1924.0,9.0,12.0,3.0
435,GENDRIN,Charles Ernest RenÃ© Marie,1898.0,12.0,15.0,14.0,0.0,Vire,1.0,14.0,...,3384.368183,NaN,NaN,1,GENDRIN,1.0,1898.0,12.0,15.0,14.0
436,GENDRIN,Charles Ernest RenÃ© Marie,1898.0,12.0,15.0,14.0,0.0,Vire,1.0,14.0,...,3384.368183,NaN,NaN,1,GENDRIN,1.0,1898.0,12.0,15.0,14.0
740,MOREAU,Madeleine Lucie,1919.0,9.0,9.0,75.0,0.0,Paris,2.0,75.0,...,8049.308110,NaN,NaN,1,MOREAU,2.0,1919.0,9.0,9.0,75.0
741,MOREAU,Madeleine Lucie,1919.0,9.0,9.0,75.0,0.0,Paris,2.0,75.0,...,8049.308110,NaN,NaN,1,MOREAU,2.0,1919.0,9.0,9.0,75.0
860,REFFIENNA,Madeleine Marie,1919.0,9.0,2.0,42.0,0.0,Saint-Chamond,2.0,42.0,...,6036.981083,NaN,NaN,1,REFFIENNA,2.0,1919.0,9.0,2.0,42.0
861,REFFIENNA,Madeleine Marie,1919.0,9.0,2.0,42.0,0.0,Saint-Chamond,2.0,42.0,...,6036.981083,NaN,NaN,1,REFFIENNA,2.0,1919.0,9.0,2.0,42.0
869,RENAUD,Lise Fannie,1914.0,1.0,13.0,75.0,0.0,Paris,2.0,75.0,...,16464.493862,NaN,NaN,1,RENAUD,2.0,1914.0,1.0,13.0,75.0
870,RENAUD,Lise Fannie,1914.0,1.0,13.0,75.0,0.0,Paris,2.0,75.0,...,16464.493862,NaN,NaN,1,RENAUD,2.0,1914.0,1.0,13.0,75.0


In [32]:
df = df.drop([242,436,741,861,870,1200,1224,1331])

In [36]:
df.to_csv('travail_final.csv')

In [35]:
df = df[[
  'Adresse_bien_vendu',
  'A_nom',
 'A_prenoms',
 'A_annee',
 'A_mois',
 'A_jour',
 'A_departement',
 'A_abroad',
 'A_ville',
 'A_sexe',
 'A-departement',
  'abroad',
 'B_nom',
 'B_sexe',
 'B_prenoms',
 'B_mois',
 'B_jour',
 'B_annee',
 'B_pays',
 'B_ville',
 'B_departement',
 'D_annee',
 'D_mois',
 'D_jour',
 'head',
 'match',
 'codebirth',
 'nbdeath',
 'reserve',
 'fichier',
 'allobs',
 'Prenom_vendeur_1',
 'Nom_vendeur_1',
 'Nom_vendeur_2',
 'Nom_acheteur_2',
 'Type_de_viager',
 'Majoration',
 'Bouquet',
 'Rente',
 'Tete_1',
 'Tete_2',
 'Commission',
 'Milliemes',
 'Standing',
 'Remarques',
 'Valeur_libre',
 'Valeur_occupee',
 'Numero',
 'Repertoire',
 'Prenom_vendeur_2',
 'Date_naissance_vendeur_1',
 'Ville_naissance_vendeur_1',
 'Departement_naissance_vendeur_1',
 'Pays_naissance_vendeur_1',
 'Date_naissance_vendeur_2',
 'Ville_naissance_vendeur_2',
 'Departement_naissance_vendeur_2',
 'Pays_naissance_vendeur_2',
 'Adresse_vendeur',
 'Nom_acheteur_1',
 'Profession_acheteur_1',
 'Date_naissance_acheteur_1',
 'Profession_acheteur_2',
 'Date_naissance_acheteur_2',
 'Adresse_acheteur',
 'Indexation_rente',
 'Date_acte_authentique',
 'Payeur_Com',
 'Chambre_de_bonnes',
 'Nb_pieces',
 'Etage',
 'Mettres_carres',
 'Ascenseur',
 'Parking_Box',
 'Cave',
 'Date_immeuble',
 'Pierre_de_taille',
 'source',
 'Ch_de_service',
 'Parking',
 'Bonus',
 'annee',
 'type',
 'quality',
 'justenom1',
 'societe',
 'vsociete',
 'asociete',
 'legasse',
 'nblegassenet',
 'reprise',
 'limite',
 'stranger1',
 'euro',
 'franc',
 'downp',
 'annuity',
 'VL',
 'VO',
 'done1',
 'nom',
 'sexe',
 'ybirth',
 'mbirth',
 'dbirth',
 'dpt_naissance']]